In [7]:
# Loading Libraries
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [8]:
# Model to device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


Files already downloaded and verified


In [10]:
# Hyper parameters
num_epochs = 10
num_classes = 10
learning_rate = 0.001

In [20]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 9, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(9),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(9, 18, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(18),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(18, 36, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(36, 72, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(72),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc1 = nn.Linear(2*2*72, 144)
        self.fc2 = nn.Linear(144, 72)
        self.fc3 = nn.Linear(72, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [21]:
model = ConvNet(num_classes).to(device)

In [22]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [23]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [100/500], Loss: 1.5318
Epoch [1/10], Step [200/500], Loss: 1.2308
Epoch [1/10], Step [300/500], Loss: 1.2011
Epoch [1/10], Step [400/500], Loss: 1.3729
Epoch [1/10], Step [500/500], Loss: 1.0219
Epoch [2/10], Step [100/500], Loss: 1.1948
Epoch [2/10], Step [200/500], Loss: 1.2189
Epoch [2/10], Step [300/500], Loss: 0.9465
Epoch [2/10], Step [400/500], Loss: 1.0950
Epoch [2/10], Step [500/500], Loss: 0.7963
Epoch [3/10], Step [100/500], Loss: 0.9311
Epoch [3/10], Step [200/500], Loss: 0.9053
Epoch [3/10], Step [300/500], Loss: 0.8831
Epoch [3/10], Step [400/500], Loss: 0.9217
Epoch [3/10], Step [500/500], Loss: 1.0468
Epoch [4/10], Step [100/500], Loss: 0.7946
Epoch [4/10], Step [200/500], Loss: 0.8319
Epoch [4/10], Step [300/500], Loss: 0.7815
Epoch [4/10], Step [400/500], Loss: 0.6713
Epoch [4/10], Step [500/500], Loss: 0.6260
Epoch [5/10], Step [100/500], Loss: 0.9167
Epoch [5/10], Step [200/500], Loss: 0.5588
Epoch [5/10], Step [300/500], Loss: 0.7010
Epoch [5/10

In [24]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 70.62 %
